# Understanding fit results: yield tables

_Valerio Ippolito - INFN Sezione di Roma_

Once you have your signal plus background model, is there anything better than looking at how it works on data?

Let's identify first of all which workspace we want to look at

In [1]:
inputFile = TString("../create_data/ws/ATLASIT_prova_combined_ATLASIT_prova_model.root");
workspaceName = TString("combined");

Since a workspace file might contain in principle multiple configurations of the statistical model, let's make sure we specify which ModelConfig we want to use:

In [2]:
modelConfigName = TString("ModelConfig");

We must also decide which data we want to look at. Real data? Asimov data? Something else?

In [3]:
dataName = TString("obsData");

This code will produce output, and output should be stored in a folder and labeled with a nickname which _tags_ the workspace (e.g. the mass of the resonance you are looking at, etc.):

In [4]:
workspaceTag = TString("my_test");
outputFolder = TString("tmp");

We want to:
- perform the fit using only few of the regions (something everyone calls either signal region[s] SR, or control regions CR)
- evaluate the output of the fit in a possibly disjoint set of regions (e.g. SR expectation after B-only fit in CR)

In [5]:
evalRegions = TString("ljets_Mbb_ge6jge4b");
fitRegions = TString("ljets_HThad_5j3b");
debugLevel = 2;

In [6]:
//!cd ../CommonStatTools; mkdir build; cd build; cmake ..; cd -

We'll use the `HistFactoryInspector` class from CommonStatTools to do this. We are running from a Jupyter notebook, so we'll have to include things manually here - but if you prefer, you can run from the command line (check `CommonStatTools/test.sh` for an example, the code is called `getHFtables.C`).

In [7]:
#include "../CommonStatTools/HistFactoryInspector.h" // must be in a separate cell, for some Jupyter reason 

In [8]:
gROOT->LoadMacro("../CommonStatTools/RooExpandedFitResult.C+");
gROOT->LoadMacro("../CommonStatTools/HistFactoryInspector.C+");


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



The big question is: do we want to fit using the S+B model, or to set all POIs to zero?

In [14]:
set_poi_zero = kTRUE

(bool) true


Here we go:

In [15]:
EXOSTATS::HistFactoryInspector hf;
hf.setDebugLevel(debugLevel);
hf.setInput(inputFile, workspaceName, modelConfigName, dataName, "", set_poi_zero); // the string should be left empty - it's the name of the range of the observable to use, something we don't need
hf.setEvalRegions(evalRegions);
hf.setFitRegions(fitRegions);

POI named mu_ttH was set to value=0 constant=1


Now, we can get the yields before and after fit:

In [16]:
hf.getYields();

 GPP : sum = 0
[#1] INFO:NumericIntegration -- RooRealIntegral::init(ttbar_ljets_Mbb_ge6jge4b_Hist_alpha_Int[obs_x_ljets_Mbb_ge6jge4b]) using numeric integrator RooBinIntegrator to calculate Int(obs_x_ljets_Mbb_ge6jge4b)
 GPP : sum = 12.8566
 GPP : sum = 0.661065
[#1] INFO:InputArguments -- The formula channelCat==channelCat::ljets_HThad_5j3b claims to use the variables (channelCat,obs_x_ljets_HThad_5j3b,obs_x_ljets_Mbb_ge6jge4b) but only (channelCat) seem to be in use.
  inputs:         channelCat==channelCat::ljets_HThad_5j3b
[#1] INFO:Minization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Minization -- RooProdPdf::getConstraints(model_ljets_HThad_5j3b) omitting term lumiConstraint as constraint term as it does not share any parameters with the other pdfs in product. To force inclusion in likelihood, add an explicit Constrain() argument for the target parameter
[#1] INFO:Minization --  Including the following constraint terms in mini

Often, you want to check the pre- and post-fit impact of uncertainties (i.e. nuisance parameters), possibly on a subset of the overall set of samples:

In [11]:
samplesForImpact = TString("ttH,ttbar,singleTop");

We do so with

In [17]:
hf.getImpacts(samplesForImpact);

[#1] INFO:InputArguments -- The formula channelCat==channelCat::ljets_HThad_5j3b claims to use the variables (channelCat,obs_x_ljets_HThad_5j3b,obs_x_ljets_Mbb_ge6jge4b) but only (channelCat) seem to be in use.
  inputs:         channelCat==channelCat::ljets_HThad_5j3b
[#1] INFO:Minization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Minization -- RooProdPdf::getConstraints(model_ljets_HThad_5j3b) omitting term lumiConstraint as constraint term as it does not share any parameters with the other pdfs in product. To force inclusion in likelihood, add an explicit Constrain() argument for the target parameter
[#1] INFO:Minization --  Including the following constraint terms in minimization: (alpha_lumiConstraint,alpha_ttXsecConstraint,alpha_JES_Scenario1_NP1Constraint,alpha_stXsecConstraint)
[#1] INFO:Minization -- The following global observables have been defined: (nom_alpha_lumi,nom_alpha_ttXsec,nom_alpha_JES_Scenario1_NP1,nom_alpha_stXs

In [13]:
//.x ../CommonStatTools/getHFtables.C(input_file, workspace_name, modelconfig_name, data_name, workspaceTag, outputFolder, eval_regions, fit_regions, doYields, doImpacts, samplesForImpact, debugLevel)